In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.linalg import block_diag
import os
output_dir = "MPC_to_NN/MPCvsNN - Sistema Compressão/output_figures"
os.makedirs(output_dir, exist_ok=True)

## Inicializando MPC

### Exemplo próprio, sistema de compressão isotérmico Dreizer (1986)

$$
\begin{bmatrix}
\dot{m}_{k+1} \\
P_{Pk+1}
\end{bmatrix}
=
\begin{bmatrix}
-8.83\times 10^{-1} & -3.72 \times 10^{-1} \\
-4.89 \times 10^{-2} & 8.20 \times 10^{1}
\end{bmatrix}
\begin{bmatrix}
\dot{m}_{k} \\
P_{Pk}
\end{bmatrix}
+
\begin{bmatrix}
1.20 & 7.14 \times 10^{-5}\\
8.73 \times 10^{-1} & 2.93 \times 10{-5}
\end{bmatrix}
\begin{bmatrix}
\alpha_k \\
N_k
\end{bmatrix}
$$

O horizonte $N = 12$ e a ação de controle deve permanecer em $[35\%, 450Hz]^\top \leq \mathbf{u} \leq [65\%, 833Hz]^\top$, logo $\mathbf{G} = I_m \otimes \begin{bmatrix}0 & 1\end{bmatrix}^\top$, $w = \begin{bmatrix}1 & 1 & \dots & 1 & 1\end{bmatrix}^\top$ e $S_u = 0$. Os parâmetros da função de custo quadrático do MPC são:

In [4]:
p = 12
nx = 2*p
nu = 2*p
scaling = 1

A = np.array([[-8.83e-1, -3.72e-1],[-4.89e-2,8.29e1]])
B = np.array([[1.20, 7.14e-5],[8.73e-1, 2.93e-5]])

A_ = np.array([[A,B],[np.zeros((nx,nx)), np.eye(nx)]])
B_ = np.array([np.zeros((nu,nu)), np.eye(nu)])

Q = np.array([6e-3, 6e-4])
R = np.array([4e-5, 2e-12])

C = np.eye(2)
C_ = np.array([C, np.zeros((2,2))])
Cu = np.array([np.zeros((2,2)), np.eye(2)])

# Psi
psi = np.array([[C_ @ A_]])
for i in range(p-1):
    psi = np.vstack((psi, psi[-1] @ A_))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 2) + inhomogeneous part.